In [ ]:
!pip install flask-ngrok

In [ ]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2022-05-03 08:23:03--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  52.5MB/s    in 0.3s    

2022-05-03 08:23:04 (52.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!/ngrok authtoken 26EUUR0GrMZSuscYNumIYtVGRr2_5KKCdnmdfXx22QibUYSKn

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
pip install wfdb

     |████████████████████████████████| 137 kB 5.2 MB/s 
     |████████████████████████████████| 11.2 MB 18.2 MB/s 
     |████████████████████████████████| 930 kB 54.6 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
pip install neurokit2

     |████████████████████████████████| 1.1 MB 5.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/ArrhythmiaUINew

/content/gdrive/MyDrive/ArrhythmiaUINew


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import pandas as pd
import tensorflow as tf
import keras
from keras.models import load_model
import wfdb
import numpy as np
from scipy.signal import butter, filtfilt

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
# app = Flask(__name__)
# path = r"/content/gdrive/My Drive/arrhythmia/models/splitonpatients.h5"
import sys

@app.route("/index")
@app.route("/")
def index():
    return render_template("index.html")

@app.route("/about")
def about():
    return render_template("about.html")

@app.route("/howitworks")
def howitworks():
    return render_template("howitworks.html")

@app.route('/model', methods=['GET', 'POST'])
def model():
    arrhythmic = 0
    condition = ""
    ratio = 0
    output = {}
    medication = ""
    
    # If a form is submitted
    if request.method == "POST":

        myfiledat = request.files["myfiledat"]
        myfilehea = request.files["myfilehea"]

        myfiledat.save(myfiledat.filename)
        myfilehea.save(myfilehea.filename)

        file_name_array = myfiledat.filename.split('.')
        record = wfdb.rdrecord(file_name_array[0])

        model = load_model("/content/gdrive/My Drive/arrhythmia/models/splitonpatients.h5")

        def resamplerecord(signal, frequency, targetfrequency):
            from wfdb import processing
            ratio = targetfrequency / frequency
            if ratio == 1.0:
              return signal

            newsignal = []
            channels = np.shape(signal)[1]
            for channel in range(channels):
              ns, _ = wfdb.processing.resample_sig(signal[:,channel], frequency, targetfrequency)
              newsignal.append(ns)
            

            return np.column_stack(newsignal)
          

        def filtersignal(signal, order, cutoff, sample_rate):
            nyquist_rate = 0.5 * sample_rate
            lowcut, highcut = cutoff[0] / nyquist_rate, cutoff[1] / nyquist_rate
            b, a = butter(order, [lowcut,highcut], btype = 'band')

            newsignal = []
            channels = np.shape(signal)[1]
            for channel in range(channels):
              ns = filtfilt(b, a, signal[:,channel])
              newsignal.append(ns)

            return np.column_stack(newsignal)

        
        from sklearn.preprocessing import StandardScaler
        import neurokit2 as nk


        frequency = record.fs
        sequences = []

        record.p_signal = resamplerecord(record.p_signal, frequency, 360)
        record.p_signal = filtersignal(record.p_signal, order = 5, cutoff = [0.5,15], sample_rate = 360)

        scaler = StandardScaler()
        record.p_signal = scaler.fit_transform(record.p_signal)

        _, rpeaks = nk.ecg_peaks(record.p_signal[:,0], sampling_rate = 360)
        rpeaks = rpeaks['ECG_R_Peaks'].tolist()

        window = 256
        for i, sample in enumerate(rpeaks):
            sequence = np.array([])
            start, end = sample - window // 2, sample + window // 2
            
            if 0 < start < end < record.p_signal.shape[0]:
            
              sequence = record.p_signal[start:end, 0]
              sequence.reshape(1, -1, 1)

              if (sequence.size > 0) :
                sequences.append(sequence)

        x = np.vstack(sequences)
        x = np.reshape(x, (x.shape[0], x.shape[1], 1))
        predictions = model.predict(x)

        
        normal = 0
        
        for (x,y), value in np.ndenumerate(predictions):
         
          if(value >= 0.5):
            arrhythmic += 1
          else:
            normal += 1

        print(f"Hello world! {arrhythmic}", file=sys.stderr)
        print("Arrhythmic", arrhythmic)

        print("Normal", normal)

        ratio = arrhythmic/(arrhythmic+normal)
        print("Ratio", ratio)


        if(ratio < 0.001):
          condition = "No arrhythmia detected"
          medication = ""

        elif (ratio < 0.002):
          condition = "Mild"
          medication = "No treatment needed but constant monitoring is needed."

        elif (ratio < 0.005):
          condition = "Moderate"
          medication = "Electric cardioversion"

        elif (ratio < 0.007):
          condition = "Severe"
          medication = "Antiarrhythmic Dosage"

        else :
          condition = "Critical"
          medication = "Immediate medical attention required."


        print("Condition", condition)
        
        # Get prediction
        output = {"arrhythmic": arrhythmic, "condition": condition, "medication": medication}
        print(output)
        
    # else:
        output = {"arrhythmic": arrhythmic, "condition": condition, "medication": medication}

    return render_template("model.html", output = output)

# Running the app


In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
